# Settings

In [1]:
import os, sys, platform, psycopg2, gc, joblib, warnings, json, pickle, datetime, time, uuid, glob
from tqdm import tqdm
from datetime import datetime
import numpy as np
import pandas as pd, numpy as np
from __future__ import annotations
from IPython.display import Markdown
from mcbs.config import settings
from mcbs.llm_api.hm_langchain_generator import LlmApiGenerator
from IPython.display import clear_output
warnings.filterwarnings('ignore')

In [2]:
from test_chat.fixed_cami_params import FixedCamiParams as cami_params
param_set = cami_params().params
param_set

{'counseling_guide': {'system_instruction': '### Role\n- You are a pet behavior counseling expert.\n- The input information provided is a summary of the pre-consultation questionnaire submitted by the dog guardian.\n- Please actively interpret and reorganize this information to produce a guide document as shown in \'### Output Content\':\n- Organize each section from an expert’s perspective so that it can be used as an actual counseling preparation document.\n- Use a chain-of-thought style and explicitly explain the rationale behind each judgment.\n\n### Output Content\n---\n#### Dog Counseling Guide\n\n##### 1. Guardian & Dog Profiling\n\n> (1) Dog\n- Basic information: name, age, sex (neutered), breed\n- Adoption method\n- Time since adoption\n- Lifestyle\n- Walking routine\n- Socialization\n- Health history\n- Level of threatening/aggressive behavior\n- Triggers and levels of alertness/anxiety\n- Temperament and disposition: interpret and describe the dog’s temperament based on the 

# Counseling Guide

In [3]:
preq_answer = {
    'animal_type': 'Dog',
    'pet_name': '랑이',
    'pet_birth_date': '2022-05-07',
    'pet_adopted_at': '2022-07-02',
    'pet_gender': 'F',
    'pet_is_neutered': True,
    'pet_breed': '진도믹스',
    'env1': "반려동물과 함께 거주하지 않음",
    'env2': '가정 분양',
    'env3': '주거용 공간(아파트, 원룸, 단독주택 등)',
    'env4': 4,
    'env5': '아니오',
    'env6': 20,
    'env7': 3,
    'env8': '없음',
    'env9': 1,
    'life_walk_day': 2,
    'life_walk_hour': 45,
    'life_walk_week': 12,
    'life_play': "가끔씩 터그 놀이 비스무리하게 놀아요. 하루에 한 번 정도 15분",
    'life_human': 18,
    'life_alone': 6,
    'life_space': ["문이 있는 플라스틱 크레이트(바리, 행복 켄넬 등)", "방석이나 매트", "몇 군데 본인이 좋아하는 자리"],
    'pref_rewards': "아빠 보호자(주보호자)의 손길",
    'med_disease': "알레르기성 피부염",
    'med_allergy': "식품에 알레르기가 있는데, 어떤 성분 때문인지는 잘 모름",
    'social_interactions': '5~10명',
    'social_trains': ["손", "앉아", "기다려", "엎드려", "돌아"],
    'fear_triggers': "바깥에서 갑작스러운 바람소리 같은 자극",
    'fear_touch': ['전반적으로 몸 전체','발', '안아서 들기', '뽀뽀하기'],
    'fear_stranger': '별다른 반응을 보이지 않는다.',
    'fear_clinic': ['낮게 으르렁거린다'] ,
    'fear_dog': '약간 그렇지 않다',
    'fear_separation': '약간 그렇다',
    'behavior_guarding': '본인이 당장 관심을 가지고 있는 모든 것. 보호자, 장난감, 간식 등 전반',
    'behavior_separation': "주보호자가 외출하려고 하면 흥분해서 달려들려고 함",
    "behavior_demand": ["보호자의 옷, 악세서리, 신체 일부를 물어서 잡아당긴다"],
    "behavior_impulsive": "약간 그렇지 않다",
    "behavior_overarousal": "전혀 그렇지 않다",
    "behavior_licking": "매우 그렇다",
    "behavior_denial": "약간 그렇지 않다",
    "behavior_aggression": "으르렁거리는데도 자꾸 만지려고 할 때, 혼자만 두고 가족들이 모두 나가려 할 때",
    "behavior_bite": "레벨2 - 개의 이빨이 상대의 피부에 닿지만, 피부에 구멍이 나지 않는다. 다만 피부에 가벼운 이빨 자국이 생기거나, 이빨이 스치면서 아주 약간의 출혈이 있을 수 있다. (상처의 길이는 2.5cm 이하로, 피부가 뚫리는 깊은 상처는 아님)",
    "counseling_needs": "외출 신호에도 차분하게 있기, 주보호자인 아빠 보호자하고만 살갑고, 다른 가족들에게는 거리를 두려고 함",
    "dcsi_label": "눈치빠른 탐험가",
    "dcsi_code": "PICA"
}
out_path = "/dog_counseling/test_chat/preq_answer.json"
if len(glob.glob(out_path)) == 0:
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(preq_answer, f, ensure_ascii=False, indent=2)
else :
    preq_answer = json.load(open(out_path, "r", encoding="utf-8"))

In [4]:
def extract_context_from_preq(info):
    info['current_date'] = datetime.now().strftime('%Y-%m-%d')
    info['pet_age'] = round((pd.to_datetime(info['current_date']) - pd.to_datetime(info['pet_birth_date'])).days/365, 1)
    info['years_current-adopted'] = round((pd.to_datetime(info['current_date']) - pd.to_datetime(info['pet_adopted_at'])).days/365, 1)
    info['weeks_adopted-birth'] = round((pd.to_datetime(info['pet_adopted_at']) - pd.to_datetime(info['pet_birth_date'])).days/7, 1)

    context = f"""
### Dog Care Counseling: Pre-Questionnaire Summary

- **Report Date**: {info['current_date']}
- **Note**: The dog participating in this counseling session will be referred to as the 'Protagonist Dog'.

---

#### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: {info['pet_name']}
- **Birth Date**: {info['pet_birth_date']} (Age: {info['pet_age']} years)
- **Adoption Date**: {info['pet_adopted_at']}
- Weeks between birth and adoption: {info['weeks_adopted-birth']} weeks
- Years living together: {info['years_current-adopted']} years
- **Breed**: {info['pet_breed']}
- **Sex**: {info['pet_gender']} (Neutered?: {info['pet_is_neutered']})
- **Adoption Method**: {info['env2']}

##### Health Record
- **Diseases (last 6 months)**: {info['med_disease']}
- **Allergies**: {info['med_allergy']}

##### Temperament & Personality
- **DCSI Personality Type**: {info['dcsi_label']} ({info['dcsi_code']})
- **Preferred Rewards**: {info['pref_rewards']}
- **Triggers for Alertness/Discomfort**: {info['fear_triggers']}
- **Body Parts Sensitive to Touch/Contact**: {info['fear_touch']}
    
##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: {info['social_interactions']}
- **Known Cues (in a calm environment, >=70% success rate)**: {info['social_trains']}
- **Reaction to Strangers Visiting Home**: {info['fear_stranger']}
- **Reaction to Vets/Nurses at the Clinic**: {info['fear_clinic']}
- **Can the dog play well with other dogs?**: {info['fear_dog']}
- **Does the dog follow the guardian wherever they go?**: {info['fear_separation']}

##### Behavioral Patterns
- **Resource Guarding Situations**: {info['behavior_guarding']}
- **Behaviors When Left Alone**: {info['behavior_separation']}
- **Demand-Seeking Behaviors (when guardian is busy)**: {info['behavior_demand']}
- **Impulsivity (reaction to curious/fun stimuli)**: {info['behavior_impulsive']}
- **Over-arousal (excessive physical reaction when excited)**: {info['behavior_overarousal']}
- **Compulsive Licking (to the point of skin damage)**: {info['behavior_licking']}
- **Reaction to Denial (intensified demands when refused)**: {info['behavior_denial']}

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- {info['behavior_aggression']}
- **Bite History & Intensity (if any)**:
- {info['behavior_bite']}

---

#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: {info['env1']}
- **Interests for Counseling/Behavior Correction**: {info['counseling_needs']}

---

#### 3. Living Environment

- **Living Space Type**: {info['env3']} (Tied-up at specific area?: {info['env5']})
- **Number of Bedrooms**: {info['env4']}
- **Dog's Personal Shelter Type**: {info['life_space']}

##### Family Composition
- **Number of Human Family Members**: {info['env7']}
- **Presence of Young or Elderly Members**: {info['env8']}
- **Number of Other Animals**: {info['env9']}

---

#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: {info['env6']}(만원)
- **Walking Routine**:
- **Frequency**: {info['life_walk_day']} times/day, {info['life_walk_week']} times/week
- **Duration**: {info['life_walk_hour']} minutes/walk
- **Other Activities (type, frequency, duration)**: {info['life_play']}
- **Time Spent with Human Family per Day**: {info['life_human']} hours
- **Time Spent Alone per Day**: {info['life_alone']} hours

---
"""
    return context

user_context = extract_context_from_preq(preq_answer)
display(Markdown(user_context))


### Dog Care Counseling: Pre-Questionnaire Summary

- **Report Date**: 2025-09-29
- **Note**: The dog participating in this counseling session will be referred to as the 'Protagonist Dog'.

---

#### 1. Protagonist Dog's Information

##### Basic Profile
- **Name**: 랑이
- **Birth Date**: 2022-05-07 (Age: 3.4 years)
- **Adoption Date**: 2022-07-02
- Weeks between birth and adoption: 8.0 weeks
- Years living together: 3.2 years
- **Breed**: 진도믹스
- **Sex**: F (Neutered?: True)
- **Adoption Method**: 가정 분양

##### Health Record
- **Diseases (last 6 months)**: 알레르기성 피부염
- **Allergies**: 식품에 알레르기가 있는데, 어떤 성분 때문인지는 잘 모름

##### Temperament & Personality
- **DCSI Personality Type**: 눈치빠른 탐험가 (PICA)
- **Preferred Rewards**: 아빠 보호자(주보호자)의 손길
- **Triggers for Alertness/Discomfort**: 바깥에서 갑작스러운 바람소리 같은 자극
- **Body Parts Sensitive to Touch/Contact**: ['전반적으로 몸 전체', '발', '안아서 들기', '뽀뽀하기']

##### Social & Environmental Experiences
- **Social Interactions during Socialization Period (people met)**: 5~10명
- **Known Cues (in a calm environment, >=70% success rate)**: ['손', '앉아', '기다려', '엎드려', '돌아']
- **Reaction to Strangers Visiting Home**: 별다른 반응을 보이지 않는다.
- **Reaction to Vets/Nurses at the Clinic**: ['낮게 으르렁거린다']
- **Can the dog play well with other dogs?**: 약간 그렇지 않다
- **Does the dog follow the guardian wherever they go?**: 약간 그렇다

##### Behavioral Patterns
- **Resource Guarding Situations**: 본인이 당장 관심을 가지고 있는 모든 것. 보호자, 장난감, 간식 등 전반
- **Behaviors When Left Alone**: 주보호자가 외출하려고 하면 흥분해서 달려들려고 함
- **Demand-Seeking Behaviors (when guardian is busy)**: ['보호자의 옷, 악세서리, 신체 일부를 물어서 잡아당긴다']
- **Impulsivity (reaction to curious/fun stimuli)**: 약간 그렇지 않다
- **Over-arousal (excessive physical reaction when excited)**: 전혀 그렇지 않다
- **Compulsive Licking (to the point of skin damage)**: 매우 그렇다
- **Reaction to Denial (intensified demands when refused)**: 약간 그렇지 않다

##### Aggression History
- **Situations Showing Threatening or Aggressive Behavior**:
- 으르렁거리는데도 자꾸 만지려고 할 때, 혼자만 두고 가족들이 모두 나가려 할 때
- **Bite History & Intensity (if any)**:
- 레벨2 - 개의 이빨이 상대의 피부에 닿지만, 피부에 구멍이 나지 않는다. 다만 피부에 가벼운 이빨 자국이 생기거나, 이빨이 스치면서 아주 약간의 출혈이 있을 수 있다. (상처의 길이는 2.5cm 이하로, 피부가 뚫리는 깊은 상처는 아님)

---

#### 2. User(Guardian) Information

- **Caregiving Role for the protagonist dog**: 반려동물과 함께 거주하지 않음
- **Interests for Counseling/Behavior Correction**: 외출 신호에도 차분하게 있기, 주보호자인 아빠 보호자하고만 살갑고, 다른 가족들에게는 거리를 두려고 함

---

#### 3. Living Environment

- **Living Space Type**: 주거용 공간(아파트, 원룸, 단독주택 등) (Tied-up at specific area?: 아니오)
- **Number of Bedrooms**: 4
- **Dog's Personal Shelter Type**: ['문이 있는 플라스틱 크레이트(바리, 행복 켄넬 등)', '방석이나 매트', '몇 군데 본인이 좋아하는 자리']

##### Family Composition
- **Number of Human Family Members**: 3
- **Presence of Young or Elderly Members**: 없음
- **Number of Other Animals**: 1

---

#### 4. Protagonist Dog's Lifestyle

- **Monthly Pet Care Cost (KRW)**: 20(만원)
- **Walking Routine**:
- **Frequency**: 2 times/day, 12 times/week
- **Duration**: 45 minutes/walk
- **Other Activities (type, frequency, duration)**: 가끔씩 터그 놀이 비스무리하게 놀아요. 하루에 한 번 정도 15분
- **Time Spent with Human Family per Day**: 18 hours
- **Time Spent Alone per Day**: 6 hours

---


In [5]:
guide_save_path = '/dog_counseling/test_chat/guide_result.pkl'
if len(glob.glob(guide_save_path)) == 0:
    llm_api_generator = LlmApiGenerator()
    guide_response = llm_api_generator.generate_message(
        query=user_context,
        params=param_set['counseling_guide'],
        message_history=None,
        tagname='guide'
        )
    joblib.dump(guide_response, guide_save_path)
else :
    guide_response = joblib.load(guide_save_path)
guide_response


'### Dog Counseling Guide\n\n##### 1. Guardian & Dog Profiling\n\n> (1) Dog\n- **Basic information**: 랑이 (Rangi), 3.4 years old, female (neutered), Jindo Mix\n- **Adoption method**: Adopted from a home breeder at 8 weeks old.\n- **Time since adoption**: 3.2 years\n- **Lifestyle**: Spends about 6 hours alone per day. Has two 45-minute walks daily. Plays a tug-like game with the guardian for about 15 minutes once a day.\n- **Walking routine**: 2 times a day, 45 minutes per session. The frequency and duration are very appropriate.\n- **Socialization**: Met 5-10 people during her socialization period, which is a relatively small number. She is described as "slightly not" good with other dogs, suggesting a lack of confidence or skill in canine social interactions.\n- **Health history**: Has allergic dermatitis and an unknown food allergy. The questionnaire indicates a "very high" level of compulsive licking, which is likely a symptom of her skin condition.\n- **Level of threatening/aggressi

# Counseling Chats

In [6]:
llm_api_generator = LlmApiGenerator()
counselor_param = param_set['counselor']
counselor_param['system_instruction'] += guide_response
turn = 0

def send_query(query, turn, params=counselor_param, llm_api_generator=llm_api_generator):
    if turn == 0:
        current_query = cami_params().initial_message
    else :
        current_query = query

    message_history = llm_api_generator.message_history
    response = llm_api_generator.generate_message(
        query=current_query,
        params=params,
        message_history=message_history,
        tagname='counselor'
    )
    llm_api_generator.update_message_history(query=current_query, response_text=response)
    return llm_api_generator, response

def show_counselor_response(response, turn):
    json_response = json.loads(response)
    front = json_response['front_message']
    back = json_response['back_thinking']
    show = f"""
### Turn {turn}

> **Back**

{back}

> **Front**

{front}
"""
    return show

llm_api_generator, response = send_query(query="", turn=turn)
show = show_counselor_response(response, turn)
display(Markdown(show))

E0000 00:00:1759129352.220671   12386 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



### Turn 0

> **Back**

The user wants me to start the counseling session. I have provided a warm greeting as Cami and presented two conversation options: problem-solving for specific topics (separation anxiety, differential affection) or a free Q&A, as per the instructions. I also asked the user to choose an option and specify any topics.

> **Front**

안녕하세요, 사색을 좋아하는 비글이자 반려견 마음 통역사 까미예요. 🐶
우리 랑이의 속마음을 깊이 읽고, 화목한 반려생활을 만들어 갈 수 있도록 함께 노력할게요! 

### 저와 어떤 대화를 시작해 볼까요?

1. 랑이가 보호자님과 떨어질 때 보이는 행동이나 가족들과의 관계 개선에 대해 **함께 깊게 논의하며** 솔루션을 찾아가는 대화도 가능하고요!
    - 보호자님 외 가족들에게 거리를 두는 행동
    - 보호자님이 외출 준비할 때 불안해하는 행동
2. 그때그때 궁금한 것을 물어보실 수 있는 자유 주제 대화도 좋아요.

**1. 솔루션 찾기**와 **2. 자유 대화** 중 어떤 대화를 나누고 싶나요? 
또 이야기 나누고 싶은 주제도 같이 전달해 주시면 좋아요.


In [14]:
turn += 1
query = """
가보자고.
"""
llm_api_generator, response = send_query(query=query, turn=turn)
show = show_counselor_response(response, turn)
display(Markdown(show))

E0000 00:00:1759129675.654283   12386 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



### Turn 8

> **Back**

The user agreed to the roadmap. I am now presenting the first step-by-step action plan for 

> **Front**

좋아요! 그럼 첫 번째 단계, '좋은 일'의 근원 되기부터 시작해 볼까요? 🐾

**🐾 Step 1: 다른 가족도 '맛있는 간식' 주는 손이 되기**

    > 랑이가 가장 좋아하는 간식! 이제 그 간식을 아빠 외 다른 가족분들이 주는 것으로 시작해 봐요. 랑이에게 다른 가족들도 '기분 좋은 일'을 가져다주는 존재라는 것을 알려주는 아주 중요한 첫걸음이랍니다.

    1. **무엇을:** 랑이가 가장 좋아하는 간식 (사람 음식은 제외하고, 평소 주는 사료나 건강한 간식으로 준비해 주세요!)
    2. **언제/어떻게:**
        - 다른 가족분들이 랑이에게 다가가지 않고, 랑이가 스스로 다가올 수 있는 거리에 앉아서 간식을 바닥에 툭 던져주세요.
        - 랑이가 간식을 먹는 동안에는 눈을 마주치지 않고, 조용히 있어 주세요. 랑이가 편안함을 느낄 수 있도록 부담을 주지 않는 것이 중요해요.
        - 하루 3~5회, 각 5분 이내로 짧게 진행합니다.
    3. **목표:** 랑이가 다른 가족분들의 존재를 '맛있는 간식'과 연결하여 긍정적인 감정을 느끼게 하는 것. 다른 가족분들이 다가와도 피하지 않고 간식을 받아먹는 것을 목표로 해요.
    4. **주의 사항:**
        - 랑이가 간식을 먹지 않고 경계한다면, 더 먼 거리에서 시도하거나 간식의 가치를 더 높여보세요.
        - 절대 랑이를 강제로 만지거나 다가가려고 하지 마세요. 랑이가 스스로 선택할 시간을 주는 것이 중요해요.
        - 아빠가 사람 음식을 주는 것은 랑이의 건강과 행동 문제에 좋지 않은 영향을 줄 수 있으니, 이 훈련 기간 동안에는 다른 가족분들과 동일하게 건강한 간식을 주시도록 가족 모두가 함께 노력해 주세요.

어떠신가요? 이 첫 번째 단계, 시작해 보실 준비 되셨나요? ✨
